### Assignment

A retailer called SpreeMart wants to improve their service experience for their customers which have different needs from their different profiles. 

As a data scientist at SpreeMart, you are asked to develop a model in Python to divide broad customers into sub-groups of customers based on some type of shared characteristics and extract useful insights from the data and your model with a given sample of customer data in the files attached as following. 

- sm_customers.csv describes the customer's information and its location.
- sm_items.csv includes data of items purchased within each order. 
- sm_orders.csv This file shows data of each order. 
- sm_payments.csv file details the order payment options. 

SpreeMart expects you to do good practice for data science including data exploratory, model development, insight interpretation and in addition, sharing the results with stakeholders by preparing a presentation for an audience of mixed technical abilities on your results and thinking process.

---